<a href="https://colab.research.google.com/github/Oh-JunTaek/sideProject/blob/main/TESTllama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import glob
from datasets import Dataset
import os

HF_API_TOKEN = ""

# Hugging Face 모델 및 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", use_auth_token=HF_API_TOKEN)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", use_auth_token=HF_API_TOKEN)

# 학습 데이터 로드 (로컬 파일에서 텍스트 파일 불러오기)
from google.colab import files
uploaded = files.upload()

all_text = []
for file_name in uploaded.keys():
    with open(file_name, "r", encoding="utf-8") as f:
        all_text.extend(f.readlines())

# 텍스트를 토큰화하여 모델 입력으로 변환
inputs = tokenizer(all_text, return_tensors="pt", max_length=512, truncation=True, padding=True, add_special_tokens=True)

# Hugging Face Datasets 객체 생성
data = [{"input_ids": input_id, "attention_mask": attention_mask}
        for input_id, attention_mask in zip(inputs['input_ids'], inputs['attention_mask'])]
dataset = Dataset.from_list(data)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",           # 결과 저장 디렉토리
    num_train_epochs=3,               # 에폭 수
    per_device_train_batch_size=1,    # 배치 크기
    per_device_eval_batch_size=1,     # 평가 배치 크기
    warmup_steps=500,                 # 워밍업 스텝
    weight_decay=0.01,                # weight decay
    logging_dir="./logs",             # 로그 저장 디렉토리
    logging_steps=10,
    save_steps=1000,                  # 체크포인트 저장 주기
    save_total_limit=3,               # 저장할 체크포인트 개수 제한
    eval_strategy="steps",            # 평가 전략
    eval_steps=1000,                  # 평가 주기
    fp16=True,                        # 16-bit 부동소수점 사용
)

# Trainer 설정
trainer = Trainer(
    model=model,                       # 모델
    args=training_args,                # 학습 인자
    train_dataset=dataset,             # 학습 데이터셋
    eval_dataset=dataset               # 평가 데이터셋
)

# 학습 시작
trainer.train()

model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_tokenizer")